# Create Your Own Visualizations!
Instructions:
1. Install tensor2tensor and train up a Transformer model following the instruction in the repository https://github.com/tensorflow/tensor2tensor.
2. Update cell 3 to point to your checkpoint, it is currently set up to read from the default checkpoint location that would be created from following the instructions above.
3. If you used custom hyper parameters then update cell 4.
4. Run the notebook!

In [1]:
import os

import tensorflow as tf
import numpy as np

from tensor2tensor import problems
from tensor2tensor.bin import t2t_decoder  # To register the hparams set
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.visualization import attention
from tensor2tensor.visualization import visualization
from tensor2tensor.utils import usr_dir
from itertools import groupby

/home/olivier/anaconda3/envs/1.5_cpu/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});

<IPython.core.display.Javascript object>

## HParams

In [8]:
# PUT THE MODEL YOU WANT TO LOAD HERE!
CHECKPOINT = "../checkpoints/word_to_phonetic_vocab/transformer-transformer_base_single_gpu-en-best_model"

In [4]:
# HParams
problem_name = 'word_to_phonetic_vocab'
data_dir = "../data_dir/en"
model_name = "transformer"
hparams_set = "transformer_base_single_gpu"

In [5]:
#Adding word to phonetic problem to the problem list
submodule_dir = "../submodule"
usr_dir.import_usr_dir(submodule_dir)

INFO:tensorflow:Importing user module submodule from path /home/olivier/Bureau/Transformer_test


## Visualization

In [10]:
visualizer = visualization.AttentionVisualizer(hparams_set, model_name, "../api/word_to_phonetic/en/files/", problem_name, beam_size=5)

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_158_512.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_158_512.targets_bottom
INFO:tensorflow:Building model body
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Transforming body output with symbol_modality_158_512.top
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Beam Decoding with beam size 5
Instructions for updating

In [11]:
tf.Variable(0, dtype=tf.int64, trainable=False, name='global_step')

sess = tf.train.MonitoredTrainingSession(
    checkpoint_dir=CHECKPOINT,
    save_summaries_secs=0,
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../checkpoints/word_to_phonetic_vocab/transformer-transformer_base_single_gpu-en-best_model/model.ckpt-11000


In [13]:
#input example
input_word = "google"
input_word = " ".join([c.upper() for c in input_word])
_, inp_text, out_text, att_mats = visualizer.get_vis_data_from_string(sess, input_word)

#Uncomment to see all attention heads and have a better understanding of what's happening inside the model
attention.show(inp_text, out_text, *att_mats)

#Removes both padding and "end of sequence" markers
inp_text = [v.strip("_") for v in inp_text if v != '<EOS>']
out_text = [v.strip("_") for v in out_text if v != '<EOS>' and v != '<pad>']

#Gets layes 0 and 4 of the "inp_out" matrices
att_matrices = np.array(attention._get_attention(inp_text, out_text, *att_mats)["inp_out"]["att"])[np.array([0,4,5]),:,:,:]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
def normalize(matrix):
    """
        input: a numpy matrix
        return: matrix with 0 mean and 1 std
    """
    return (matrix - np.mean(matrix))/np.std(matrix)

#Sum all attention heads
sum_all_head = np.sum(att_matrices, axis=1)

#Sum layers 0 and 4
sum_all_layers = normalize(np.sum(sum_all_head,axis=0)[:len(out_text), :len(inp_text)])

In [15]:
#Base threshold
#fr : 0.75
#es : 0.4
if len(out_text) > 4:
    threshold = 0.75
else : 
    threshold = 0

#While we have too many silent_letters detected
while(True):
    #Gets the silent_letters indices
    #We consider that a letter is silent if its attention value is below mean attention + threshold * std attention
    silent_letters_idx = [i for i, idx in enumerate(np.argmax(sum_all_layers, axis = 0)) 
                          if sum_all_layers[idx, i] < np.mean(sum_all_layers[idx,:]) 
                          + threshold*np.std(sum_all_layers[idx,:])]
    #Reduces threshold if too many silent letters are detected
    #Can happen in french when we have 3 lettres graphemes
    if len(silent_letters_idx) > 1/3 * len(inp_text):
        threshold -= 0.1
    else:
        break

#Creates the phoneme attribution list
phon_list = np.array(out_text)[np.argmax(sum_all_layers, axis = 0)]
phon_list[silent_letters_idx] = "#" #"#" is our encoding for silent letters
phon_list = phon_list.tolist() #needed for the += just below

#Checks if all the phonemes are attributed and if they are only present the correct number of time in the list
#If not, the phoneme is concatenated to its most probable neighbor
#and the least probable phoneme is replaced by a silent letter (this can happen for small datasets)
for i, phon in enumerate(out_text):
    if phon not in phon_list:
        phon_list[np.argmax(sum_all_layers[i,:])] += phon
    
#     test = np.where(np.array(phon_list) == phon)[0]
#     if len(test > 1):
#         phon_list[np.max(test)] = "%"

##NOT WORKING PROPERLY

#Creates the g_p tupple list
g_p = [(l, phon_list[i]) for i, l in enumerate(inp_text)]

#Creates the final g_p mapping
mapping = []
for phon, letters in groupby(g_p, lambda x: x[1]):
    graph = "".join([letter[0] for letter in letters])
    mapping.append(graph + "-" + phon)
print(mapping)

['G-G', 'OO-UW1', 'G-G', 'L-AH0L', 'E-#']


In [16]:
def recurent_check(normalized_value_row, idx_row, idx_last_phon, out_text, j):
    print(normalized_value_row, np.std(normalized_value_row))
    if normalized_value_row[j] > 0.75*np.std(normalized_value_row):
        if(idx_row[j] == 0 or idx_row[j] >= idx_last_phon):
            return idx_row[j], out_text[idx_row[j]]
        else:
            return recurent_check(normalized_value_row, idx_row, idx_last_phon, out_text, j+1)
    else:
        return -1, "$"


phon_list = []
idx_last_phon = 0

idx_sorted_phon_per_letter = np.argsort(-sum_all_layers, axis=0)[:,:].transpose()
value_sorted_phon_per_letter = -np.sort(-sum_all_layers, axis=0)[:,:].transpose()

for i, idx in enumerate(idx_sorted_phon_per_letter[:,0]):
    normalized_value_row = value_sorted_phon_per_letter[i,:]
    idx_row = idx_sorted_phon_per_letter[i,:]
    
    idx_phon, phon = recurent_check(normalized_value_row, idx_row, idx_last_phon, out_text, 0)
    
    phon_list.append(phon)
    idx_last_phon = idx_phon
    
#Creates the g_p tupple list
g_p = [(l, phon_list[i]) for i, l in enumerate(inp_text)]

#Creates the final g_p mapping
mapping = []
for phon, letters in groupby(g_p, lambda x: x[1]):
    graph = "".join([letter[0] for letter in letters])
    mapping.append(graph + "-" + phon)
print(mapping)

[ 3.12843119 -0.52949637 -0.64148888 -0.68603657 -0.69181073] 1.5073829940093453
[ 1.43735743 -0.44006676 -0.56597202 -0.61989395 -0.75279828] 0.8189891797786931
[ 1.1668267  -0.40232972 -0.50103401 -0.51685242 -0.52696645] 0.6629370768237967
[ 1.21970625 -0.38671595 -0.42139625 -0.4904222  -0.59952313] 0.6815742351822552
[ 1.99302559  1.91403442  1.00027217 -0.44271926 -0.53762497] 1.0988191195153327
[-0.00431167 -0.3546456  -0.49054736 -0.50776283 -0.74923838] 0.24418709060523336
['G-G', 'OO-UW1', 'G-G', 'L-AH0', 'E-$']


In [17]:
g_p = []
base_threshold = 1

idx_sorted_letter_per_phon = np.argsort(-sum_all_layers, axis=1)[:,:]
value_sorted_letter_per_phon = -np.sort(-sum_all_layers, axis=1)[:,:]

for i, phon in enumerate(out_text):
    normalized_value_row = value_sorted_letter_per_phon[i,:]
    idx_row = idx_sorted_letter_per_phon[i,:]
    threshold = base_threshold
    while(True):
        letters = np.array(inp_text)[idx_row[np.where(normalized_value_row > threshold * np.std(normalized_value_row))]]
        if len(letters) == 0:
            threshold -= 0.1
        else:
            break
    for l in letters:
        g_p.append((l, phon))
        
for i, l in enumerate(inp_text):
    if i == len(g_p):
        g_p.append((l, "$"))
    elif g_p[i][0] != l:
        g_p.insert(i, (l, "$"))

mapping = []
for phon, letters in groupby(g_p, lambda x: x[1]):
    graph = "".join([letter[0] for letter in letters])
    mapping.append(graph + "-" + phon)
print(mapping)

['G-G', 'OO-UW1', 'GL-G', 'E-$', 'L-AH0', 'L-L']


In [18]:
phon_list = []
idx_sorted_phon_per_letter = np.argsort(-sum_all_layers, axis=0)[:,:].transpose()
value_sorted_phon_per_letter = -np.sort(-sum_all_layers, axis=0)[:,:].transpose()

for i, idx in enumerate(idx_sorted_phon_per_letter[:,0]):
    normalized_value_row = value_sorted_phon_per_letter[i,:]
    idx_row = idx_sorted_phon_per_letter[i,:]
    
    phon = np.array(out_text)[idx_row[np.where(normalized_value_row > 1.2 * np.std(normalized_value_row))]]
    if len(phon) > 0:
        phon_list.append("".join(phon))
    else:
        phon_list.append("$")

#Creates the g_p tupple list
g_p = [(l, phon_list[i]) for i, l in enumerate(inp_text)]

#Creates the final g_p mapping
mapping = []
for phon, letters in groupby(g_p, lambda x: x[1]):
    graph = "".join([letter[0] for letter in letters])
    mapping.append(graph + "-" + phon)
print(mapping)

['G-G', 'OO-UW1', 'G-G', 'L-AH0L', 'E-$']


In [19]:
def get_gp_match(i, txt, pred, attentionMatrix, treshold = 0.5):
    temp = []
    previous = 0
    if txt[i] != ['']:
        if len(txt[i]) > 1 and len(pred[i]) > 1:
            for j in range(len(pred[i])):
                graph = ""
                if j+1 < len(attentionMatrix[i]):            
                    end = attentionMatrix[i][j+1].argmax()
                    while ((previous < end or previous == attentionMatrix[i][j].argmax())
                           and (attentionMatrix[i][j+1][previous] < treshold or attentionMatrix[i][j][previous] > treshold)):

                        graph += txt[i][previous]
                        previous +=1

                else:           
                    while (previous < len(attentionMatrix[i][j]) 
                           and (attentionMatrix[i][j][previous] > treshold
                           or previous == attentionMatrix[i][j].argmax()+1
                           or previous == attentionMatrix[i][j].argmax()+2
                           or previous == attentionMatrix[i][j].argmax()+3)):
                        graph += txt[i][previous]
                        previous +=1                      
                g = graph
                p = pred[i][j]
                gp = ("-").join([g,p])
                temp.append(gp)

                if g == "":
                    shift(temp,j)

        elif len(txt[i]) == 1:
            g = txt[i][0]
            p = pred[i][0]
            gp = ("-").join([g,p])
            temp.append(gp)

        elif len(pred[i]) == 1:
            g = ("").join(txt[i])
            p = pred[i][0]
            gp = ("-").join([g,p])
            temp.append(gp)

    return temp

In [28]:
visualizer.encoders['inputs'].decode([2])

'N'

In [37]:
dump_vocab = [visualizer.encoders['inputs'].decode([i])for i in range(visualizer.encoders['inputs'].vocab_size)]

In [38]:
dump_vocab

['<pad>',
 '<EOS>',
 'N',
 'S',
 'R',
 'E',
 'T',
 'L',
 'A',
 'I',
 'D',
 'O',
 'AH0',
 'M',
 'K',
 'B',
 'P',
 'G',
 'C',
 'Z',
 'U',
 'IH0',
 'H',
 'F',
 'ER0',
 'IY0',
 'V',
 'Y',
 'EH1',
 'W',
 'AA1',
 'AE1',
 'IH1',
 'IY1',
 'NG',
 'HH',
 'EY1',
 ' )',
 'SH',
 ' ( ',
 'OW1',
 '1',
 'OW0',
 'AO1',
 " ' ",
 'AY1',
 'AH1',
 'UW1',
 'JH',
 'AA0',
 'CH',
 'ER1',
 'IH2',
 'EH2',
 'EY2',
 'AE2',
 'AY2',
 'AA2',
 'EH0',
 'TH',
 'J',
 'IY2',
 'OW2',
 'AW1',
 'X',
 'UW0',
 'AO2',
 'AE0',
 'UH1',
 'AO0',
 'Q',
 'AY0',
 'UW2',
 'AH2',
 ' - ',
 'EY0',
 'OY1',
 " '",
 'AW2',
 'ER2',
 'DH',
 'ZH',
 '2',
 'UH2',
 'AW0',
 'UH0',
 'OY2',
 '3',
 'OY0',
 ' _ ',
 " ' ( ",
 ' .',
 ' . ',
 " . ' ",
 ' . ( ',
 '0',
 '4',
 '{ ',
 '8',
 '} ',
 '9',
 '6',
 '5',
 ' \\ ',
 '7',
 '}',
 '{',
 'u',
 'p',
 'd',
 'a',
 '',
 '\\',
 'Z',
 'Y',
 'X',
 'W',
 'V',
 'U',
 'T',
 'S',
 'R',
 'Q',
 'P',
 'O',
 'N',
 'M',
 'L',
 'K',
 'J',
 'I',
 'H',
 'G',
 'F',
 'E',
 'D',
 'C',
 'B',
 'A',
 '>',
 '<',
 ';',
 '9',
 '8',
